In [2]:
import scipy.optimize as sco
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(777)
from datetime import *

In [3]:
def portfolio_annualised_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns*weights ) *252
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return std, returns

def random_portfolios(num_portfolios, mean_returns, cov_matrix, risk_free_rate):
    results = np.zeros((3,num_portfolios), dtype=object)
    weights_record = []
    for i in range(num_portfolios):
        weights = np.random.random(len(tickers))
        weights = np.sum(weights)
        weights_record.append(weights)
        portfolio_std_dev, portfolio_return = portfolio_annualised_performance(
            weights, mean_returns, cov_matrix)
        # func(results[0],i) = portfolio_std_dev
        results[0,i] = portfolio_std_dev
        results[1,i] = portfolio_return
        results[2,i] = ((portfolio_return - risk_free_rate) / portfolio_std_dev)
    return results, weights_record

def display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results, weights = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)
    rr = np.array(results[2])
    max_sharpe_idx = rr.argmax()
    sdp, rp = results[0,max_sharpe_idx], results[1,max_sharpe_idx]
    max_sharpe_allocation = pd.DataFrame(weights[max_sharpe_idx],index=PT.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    min_vol_idx = np.argmin(results[0])
    sdp_min, rp_min = results[0,min_vol_idx], results[1,min_vol_idx]
    min_vol_allocation = pd.DataFrame(weights[min_vol_idx],index=PT.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Allocation\n")
    print ("Annualised Return:", round(rp,2))
    print ("Annualised Volatility:", round(sdp,2))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Allocation\n")
    print ("Annualised Return:", round(rp_min,2))
    print ("Annualised Volatility:", round(sdp_min,2))
    print ("\n")
    print (min_vol_allocation)
    plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')
    plt.title('Simulated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.8)


def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    p_var, p_ret = portfolio_annualised_performance(weights, mean_returns, cov_matrix)
    return -(p_ret - risk_free_rate) / p_var

def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def portfolio_volatility(weights, mean_returns, cov_matrix):
    return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[0]

def min_variance(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))

    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)

    return result

def efficient_return(mean_returns, cov_matrix, target):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)

    def portfolio_return(weights):
        return portfolio_annualised_performance(weights, mean_returns, cov_matrix)[1]

    constraints = ({'type': 'eq', 'fun': lambda x: portfolio_return(x) - target},
                   {'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0,1) for asset in range(num_assets))
    result = sco.minimize(portfolio_volatility, num_assets*[1./num_assets,],
                          args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def efficient_frontier(mean_returns, cov_matrix, returns_range):
    efficients = []
    for ret in returns_range:
        efficients.append(efficient_return(mean_returns, cov_matrix, ret))
    return efficients

def display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate):
    results = random_portfolios(num_portfolios,mean_returns, cov_matrix, risk_free_rate)
    
    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp, rp = portfolio_annualised_performance(max_sharpe['x'], mean_returns, cov_matrix)
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=PT.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    max_sharpe_allocation

    min_vol = min_variance(mean_returns, cov_matrix)
    sdp_min, rp_min = portfolio_annualised_performance(min_vol['x'], mean_returns, cov_matrix)
    min_vol_allocation = pd.DataFrame(min_vol.x,index=PT.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Allocation\n")
    print ("Annualised Return:", round(rp,2))
    print ("Annualised Volatility:", round(sdp,2))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Allocation\n")
    print ("Annualised Return:", round(rp_min,2))
    print ("Annualised Volatility:", round(sdp_min,2))
    print ("\n")
    print (min_vol_allocation)
    
    plt.figure(figsize=(10, 7))
    plt.scatter(results[0,:],results[1,:],c=results[2,:],cmap='YlGnBu', marker='o', s=10, alpha=0.3)
    plt.colorbar()
    plt.scatter(sdp,rp,marker='*',color='r',s=500, label='Maximum Sharpe ratio')
    plt.scatter(sdp_min,rp_min,marker='*',color='g',s=500, label='Minimum volatility')

    target = np.linspace(rp_min, 0.32, 50)
    efficient_portfolios = efficient_frontier(mean_returns, cov_matrix, target)
    plt.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.', color='black', label='efficient frontier')
    plt.title('Calculated Portfolio Optimization based on Efficient Frontier')
    plt.xlabel('annualised volatility')
    plt.ylabel('annualised returns')
    plt.legend(labelspacing=0.8)
    
def display_ef_with_selected(mean_returns, cov_matrix, risk_free_rate, destination, saveName):
    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    sdp, rp = portfolio_annualised_performance(max_sharpe['x'],mean_returns, cov_matrix)
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=PT.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    max_sharpe_allocation.to_pickle(destination + saveName + "_max_sharpeRatio_allocation.pkl")
    min_vol = min_variance(mean_returns, cov_matrix)
    sdp_min, rp_min = portfolio_annualised_performance(min_vol['x'], mean_returns, cov_matrix)
    min_vol_allocation = pd.DataFrame(min_vol.x,index=PT.columns,columns=['allocation'])
    min_vol_allocation.allocation = [round(i*100,2)for i in min_vol_allocation.allocation]
    min_vol_allocation = min_vol_allocation.T
    min_vol_allocation.to_pickle(destination + saveName + "_min_volatility_allocation.pkl")
    an_vol = np.std(returns) * np.sqrt(252)
    an_rt = mean_returns * 252
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Allocation\n")
    print ("Annualised Return:", round(rp,2))
    print ("Annualised Volatility:", round(sdp,2))
    print ("\n")
    print (max_sharpe_allocation)
    print ("-"*80)
    print ("Minimum Volatility Portfolio Allocation\n")
    print ("Annualised Return:", round(rp_min,2))
    print ("Annualised Volatility:", round(sdp_min,2))
    print ("\n")
    print (min_vol_allocation)
    print ("-"*80)
    print ("Individual Stock Returns and Volatility\n")
    for i, txt in enumerate(PT.columns):
        print (txt,":","annuaised return",round(an_rt[i],2),
               ", annualised volatility:",round(an_vol[i],2))
    print ("-"*80)
    
    # fig, ax = plt.subplots(figsize=(10, 7))
    # ax.scatter(an_vol,an_rt,marker='o',s=200)
    # for i, txt in enumerate(PT.columns):
    #     ax.annotate(txt, (an_vol[i],an_rt[i]), 
    #                 xytext=(10,0), textcoords='offset points')
    # ax.scatter(sdp,rp,marker='*',color='r',s=500,
    #            label='Maximum Sharpe ratio')
    # ax.scatter(sdp_min,rp_min,marker='*',
    #            color='g',s=500, label='Minimum volatility')
    # target = np.linspace(rp_min, 0.34, 50)
    # efficient_portfolios = efficient_frontier(mean_returns, cov_matrix, target)
    # ax.plot([p['fun'] for p in efficient_portfolios], target, linestyle='-.',
    #         color='black', label='efficient frontier')
    # ax.set_title('Portfolio Optimization with Individual Stocks')
    # ax.set_xlabel('annualised volatility')
    # ax.set_ylabel('annualised returns')
    # ax.legend(labelspacing=0.8)
    # plt.show()

In [4]:
if __name__ == '__main__':
    # saveName = 'sample_data' 'sp500' 'dow' 'nasdaq' 'sample'
    # saveName = 'broker_pos_data' 'roth_pos_data', 'moveOn_pos_data', 'potential_pos_data'
    saveName = 'sp500'
    start, end = datetime(2020, 6, 1), datetime.now()
    start_date, end_date = start, end
    p = "/home/gordon/work/Where-To-Put-Your-Money-In-Hard-Times/data/"

    path = p + 'raw/' + saveName + '_10y_1d.pkl'
    PT_data = pd.read_pickle(path)
    PT = pd.DataFrame(PT_data)
    tickers = list(PT.columns)
    
    returns = PT.pct_change()
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    num_portfolios = 25000
    risk_free_rate = 0.0178
    destination = "/home/gordon/work/Where-To-Put-Your-Money-In-Hard-Times/data/processed/"

    # display_simulated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)
    # display_calculated_ef_with_random(mean_returns, cov_matrix, num_portfolios, risk_free_rate)
    display_ef_with_selected(mean_returns, cov_matrix, risk_free_rate, destination, saveName)

--------------------------------------------------------------------------------
Maximum Sharpe Ratio Portfolio Allocation

Annualised Return: 0.67
Annualised Volatility: 0.16


              A  AAL  AAP  AAPL  ABBV  ABC  ABMD  ABT  ACN  ADBE  ADI  ADM  \
allocation  0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0   

            ADP  ADSK  AEE  ...   WST   WU   WY  WYNN  XEL  XLNX  XOM  XRAY  \
allocation  0.0   0.0  0.0  ...  0.38  0.0  0.0   0.0  0.0   0.0  0.0   0.0   

            XRX  XYL  YUM  ZBH  ZBRA  ZION  ZTS  
allocation  0.0  0.0  0.0  0.0   0.0   0.0  0.0  

[1 rows x 505 columns]
--------------------------------------------------------------------------------
Minimum Volatility Portfolio Allocation

Annualised Return: 0.27
Annualised Volatility: 0.11


              A  AAL  AAP  AAPL  ABBV  ABC  ABMD  ABT  ACN  ADBE  ADI  ADM  \
allocation  0.0  0.0  0.0   0.0   0.0  0.0   0.0  0.0  0.0   0.0  0.0  0.0   

            ADP  ADSK  AEE  ...   WST   WU   WY  

In [23]:
import pandas
import numpy as np
from pandas.io.pickle import read_pickle

class Look_At_Optimized_Portfolios(object):
    def __init__(self, key):
        path = '/home/gordon/work/Where-To-Put-Your-Money-In-Hard-Times/data/processed/'
        self.path1 = (path + key + '_max_sharpeRatio_allocation.pkl')
        self.path2 = (path + key + '_min_volatility_allocation.pkl')

    def viz(self):
        df = read_pickle(self.path1)
        fd = read_pickle(self.path2)
        a = np.where(df.T['allocation'] > 0.0)
        b = np.where(fd.T['allocation'] > 0.0)
        return df.T, fd.T, a, b



if __name__ == '__main__':
    # saveName = 'sample_data' 'sp500' 'dow' 'nasdaq' 'sample'
    # saveName = 'broker_pos_data' 'roth_pos_data', 'moveOn_pos_data', 'potential_pos_data'    
    key = 'dow'
    x = Look_At_Optimized_Portfolios(key)
    df, fd, a, b = x.viz()
    print('\nMaximum Sharpe Ratio Portfolio:')
    df.iloc[a]
    print('\n\nMinimum Volatility Portfolio')
    fd.iloc[b]


Maximum Sharpe Ratio Portfolio:


,allocation
AAPL,18.96
HD,19.34
MSFT,3.14
NKE,11.04
UNH,17.85
V,23.25
WMT,6.42




Minimum Volatility Portfolio


,allocation
AAPL,0.92
JNJ,16.23
KO,12.93
MCD,15.43
MRK,3.67
PG,12.40
VZ,21.03
WMT,17.40
